<a href="https://colab.research.google.com/github/anna1904/AjaxRequist/blob/master/NLP_model_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
train_df = pd.read_csv('/home/dataset/train.csv')
test_df = pd.read_csv('/home/dataset/test.csv')

**DATA INVESTIGATION**

In [24]:
len(train_df)
train_df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [4]:
len(test_df)

7

In [26]:
train_df.isnull().sum()

id                   0
url_legal         2004
license           2004
excerpt              0
target               0
standard_error       0
dtype: int64

I will just drop 'url_legal' and 'license' columns

In [28]:
train_df.drop(['url_legal' , 'license'] , axis=1 , inplace = True)

In [29]:
train_df.head()

,id,excerpt,target,standard_error
0,c12129c31,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,Once upon a time there were Three Bears who li...,0.247197,0.510845


**PREPROCESSING**

In [34]:
import re

In [31]:
def preprocess(text):
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    text = text.str.replace(regex_pat, ' ')
    
    # removal of punctuations and numbers
    punc_remove = text.str.replace("[^a-zA-Z]", " ")
    
    # remove whitespace with a single space
    new_text=punc_remove.str.replace(r'\s+', ' ')
    
    # remove leading and trailing whitespace
    new_text=new_text.str.replace(r'^\s+|\s+?$','')
    
    # replace normal numbers with numbr
    new_text=new_text.str.replace(r'\d+(\.\d+)?','numbr')
    
    # removal of capitalization
    text_lower = new_text.str.lower()
    
    # tokenizing
    tokenized_text = text_lower.apply(lambda x: x.split())
    
    for i in range(len(tokenized_text)):
        tokenized_text[i] = ' '.join(tokenized_text[i])
        texts_p= tokenized_text
    
    return texts_p

In [35]:
text = train_df.excerpt
preprocessed_text = preprocess(text)

train_df['preprocessed_excerpt'] = preprocessed_text

<ipython-input-31-a1364582d8e6>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  punc_remove = text.str.replace("[^a-zA-Z]", " ")
<ipython-input-31-a1364582d8e6>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  new_text=punc_remove.str.replace(r'\s+', ' ')
<ipython-input-31-a1364582d8e6>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  new_text=new_text.str.replace(r'^\s+|\s+?$','')
<ipython-input-31-a1364582d8e6>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  new_text=new_text.str.replace(r'\d+(\.\d+)?','numbr')


In [36]:
train_df.head()

,id,excerpt,target,standard_error,preprocessed_excerpt
0,c12129c31,When the young people returned to the ballroom...,-0.340259,0.464009,when the young people returned to the ballroom...
1,85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,all through dinner time mrs fayre was somewhat...
2,b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,as roger had predicted the snow departed as qu...
3,dd1000b26,And outside before the palace a great garden w...,-1.054013,0.450007,and outside before the palace a great garden w...
4,37c1b32fb,Once upon a time there were Three Bears who li...,0.247197,0.510845,once upon a time there were three bears who li...


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 119.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.8 MB/s eta 0:00:00


In [5]:
import transformers
transformers.__version__
# !python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

'4.27.1'

In [9]:
import numpy as np
from transformers import AutoTokenizer
import torch
import tqdm
import gc
import pickle
import random

Check either GPU is available (not for google colab):

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ', device.type)

Device:  cpu


Set up seed for replication results: 

In [15]:
def set_random_seed(seed, using_cuda = False): #but default using CPU
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if using_cuda:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.cuda.manual_seed(seed)
seed = 1
set_random_seed(seed)

Feature engineering:

In [7]:
def feature_engineering(df):
    def total_words(x):
        return len(x.split(" "))

    def total_unique_words(x):
        return len(np.unique(x.split(" ")))

    def total_charactors(x):
        x = x.replace(" ", "")
        return len(x)

    def total_sentence(x):
        x = x.replace("!", "[end]").replace("?", "[end]").replace(".", "[end]")
        return len(x.split("[end]"))

    df_ret = df[["id", "excerpt", "target", "standard_error"]]
    excerpt = df["excerpt"].values
    df_ret["total_words"] = [total_words(x) for x in excerpt]
    df_ret["total_unique_words"] = [total_unique_words(x) for x in excerpt]
    df_ret["total_characters"] = [total_charactors(x) for x in excerpt]
    df_ret["total_sentence"] = [total_sentence(x) for x in excerpt]

    df_ret["div_sentence_characters"] = df_ret["total_sentence"] / df_ret["total_characters"]
    df_ret["div_sentence_words"] = df_ret["total_sentence"] / df_ret["total_words"]
    df_ret["div_characters_words"] = df_ret["total_characters"] / df_ret["total_words"]
    df_ret["div_words_unique_words"] = df_ret["total_words"] / df_ret["total_unique_words"]

    for i, word in enumerate(["!", "?", "(", ")", "'", '"', ";", ".", ","]):
        df_ret[f"count_word_special_{i}"] = [x.count(word) for x in excerpt]

    return df_ret.fillna(0)

In [16]:

cfg ={}
ARCH_PATH = DISTILBERT
cfg['train'] = {'n_folds': 5}
cfg['tokenizer'] ={'name': ARCH_PATH, 
                   'max_length': 210}

In [21]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [22]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor()
print(w2v2_extractor)

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)